In [ ]:
%pip install accelerate
%pip install sacrebleu

import torch
from transformers import pipeline
from tqdm import tqdm
import json
from sacrebleu.metrics import BLEU



# save the challenge set in json format as lists

references = []
sources = []

with open("Challenge_set-v2hA.json", 'r', encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        references.append(data["reference"])
        sources.append(data["source"])

# remove all the punctuations from the references and sources

import string

def remove_punctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation))

references_cleaned = [remove_punctuations(ref) for ref in references]
sources_cleaned = [remove_punctuations(src) for src in sources]

In [2]:
# LLaMA2-13b model

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("SnypzZz/Llama2-13b-Language-translate")
model = AutoModelForSeq2SeqLM.from_pretrained("SnypzZz/Llama2-13b-Language-translate")

In [13]:
# translate the sources to french

translated_sources = []

for src in sources_cleaned:
    model_inputs = tokenizer(src, return_tensors="pt", padding=True, truncation=True)
    generated_tokens = model.generate(**model_inputs, forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"])
    output = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    translated_sources += output
    print("Translated source: ", output)

Translated source:  ['Les appels répétés de sa mère auraient dû nous avertir']
Translated source:  ['Le bruit soudain dans les chambres supérieures aurait dû nous alerter']
Translated source:  ['Leur incapacité répétée de déclarer le problème aurait dû nous avertir']
Translated source:  ['Elle a demandé à son frère de ne pas être arrogant.']
Translated source:  ['Elle a promis à son frère de ne pas être arrogant.']
Translated source:  ['Elle a promis à son médecin de rester actif après sa retraite.']
Translated source:  ["Ma mère a promis à mon père d'être plus prudent sur la route"]
Translated source:  ['La femme était très haute et extrêmement forte.']
Translated source:  ['Leurs politiciens étaient plus ignorants que stupides']
Translated source:  ['Nous avons crié un insulte et nous avons quitté brusquement.']
Translated source:  ['Le chat et le chien devraient être surveillés']
Translated source:  ['Mon père et mon frère seront heureux demain']
Translated source:  ['Mon livre et m

In [15]:
# BLEU score

bleu = BLEU()
bleu_score = bleu.corpus_score(translated_sources, references_cleaned)
print("BLEU score: ", bleu_score.score)

BLEU score:  0.44420544193465444
